In [5]:
import scipy.stats as sps
from sklearn.model_selection import train_test_split
import gc
from dateutil.parser import *
import pandas as pd
import numpy as np
import os
import yt.wrapper as yt 
os.chdir('/Users/lunin-dv/Desktop/Library/')
import importlib
import my_library as lib
import operator
import re
importlib.reload(lib)
os.chdir('/Users/lunin-dv/Desktop/data/')

In [6]:
def copy_before(table_name):
    if yt.row_count("//home/cloud_analytics/emailing/sender/" + table_name) == 0:
        yt.copy("//home/cloud_analytics/emailing/sender_copy/" + table_name,
                "//home/cloud_analytics/emailing/sender/" + table_name,
                force=True)
        print('saved')
copy_before("Go-to-paid-trial-usage-OLD")
copy_before("Go-to-paid-trial-usage")

# 0. Общая информация

## Drop_list

In [7]:
drop_req = """
SELECT
    *
FROM "//home/cloud_analytics/emailing/sender/drop_list"
FORMAT TabSeparatedWithNames
"""
dropped_emails = lib.execute_query(drop_req)
dropped_emails = set(dropped_emails['email'])

## Cube

In [8]:
req = """
SELECT
    DISTINCT
    billing_account_id,
    lower(user_settings_email) as email,
    puid,
    event_time as console_regstration_date,
    multiIf(first_ba_created_datetime = '0000-00-00 00:00:00', '',
            first_ba_created_datetime) as ba_created,
    
    if (first_first_paid_consumption_datetime == '0000-00-00 00:00:00', '',
        first_first_paid_consumption_datetime) as first_first_paid_consumption_datetime,
    ba_usage_status as usage_status,
    ba_state,
    is_isv,
    mail_feature,
    mail_info,
    mail_promo,
    language,
    modulo(toInt64(puid), 100) as group_index,
    toDate(NOW()) - toDate(ba_created) as days_since_ba_created
FROM "//home/cloud_analytics/cubes/acquisition_cube/cube" as a
ANY LEFT JOIN(
    SELECT
        passport_uid as puid,
        user_settings_language as language
    FROM "//home/cloud_analytics/import/iam/cloud_owners_history"
) as b
ON a.puid == b.puid
WHERE
     (
        (event == 'cloud_created' and  billing_account_id in
         (SELECT DISTINCT billing_account_id 
          FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
          WHERE event == 'cloud_created')
         )
      OR
        (event == 'ba_created' and  billing_account_id not in
         (SELECT DISTINCT billing_account_id 
          FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
          WHERE event == 'cloud_created')
        )
    )
    AND billing_account_id != ''
    AND puid != ''
FORMAT TabSeparatedWithNames
"""
main_df = lib.execute_query(req)

# 1. Обновление

In [9]:
def update_current_table(table_name, path, tables_to_update):
    full_path = path + "/" + table_name
    req = f"""
    SELECT
        *
    FROM "{full_path}"
    FORMAT TabSeparatedWithNames
    """
    old_df = lib.execute_query(req)
    if 'sended_mails' in old_df.columns:
        old_df['sended_mails'] = old_df['sended_mails'].apply(
            lambda x: x.replace("\\", ""))
    old_df['is_dropped'] = (old_df['email'].isin(dropped_emails)).astype(int)
    if "OLD" in table_name:
        old_df = old_df[old_df['is_dropped'] == 0]
    for table, key in tables_to_update:
        old_df = pd.merge(old_df,
                          table, on=key, suffixes=('', '_new'), how='left')
        
    new_cols = []
    for column in old_df.columns:
        if "_new" == column[-4:]:
            new_cols.append(column)
            old_column = column[:-4]
            old_df[old_column] = old_df[[old_column, column]].apply(
            lambda x: x[column] if not pd.isnull(x[column]) else x[old_column], axis=1
            )
    old_df.drop(columns=new_cols, inplace=True)
    lib.save_table(table_name, path, old_df)

In [10]:
tables_to_update = [(main_df, "billing_account_id")]

In [11]:
update_current_table("Go-to-paid-trial-usage", "//home/cloud_analytics/emailing/sender", 
                     tables_to_update)

In [12]:
update_current_table("Go-to-paid-trial-usage-OLD", 
                     "//home/cloud_analytics/emailing/sender", 
                     tables_to_update)

# 2. Добавление

## old Marketo

In [13]:
marketo_previous_req = """
SELECT
    DISTINCT
    billing_account_id,
    email
FROM "//home/cloud_analytics/cubes/emailing_events/emailing_events"
WHERE event == 'add_to_nurture_stream'
AND lower(program_name) like '%go-to-paid%'
FORMAT TabSeparatedWithNames
"""
marketo_prev_df = lib.execute_query(marketo_previous_req)

## old Upsell

In [14]:
def old_upsell(table):
    path = '//home/cloud_analytics/emailing/sender/' + table
    req = f"""
    SELECT
        DISTINCT
        billing_account_id,
        email
    FROM "{path}"
    FORMAT TabSeparatedWithNames
    """
    upsell_df = lib.execute_query(req)
    return upsell_df

In [15]:
tables = ["CLOUDANA-724_company",
          "CLOUDANA-724_individual",
          "CLOUDANA-834_Upsell-3_paid",
          "CLOUDANA-834_Upsell-3_trial",
          "CLOUDANA-794_mdb_on_vm"]
upsells = []
for table in tables:
    upsells.append(old_upsell(table))

upsell_df = lib.concatenate_tables(upsells)

## old in program


In [16]:
req = """
SELECT
    DISTINCT
        email,
        billing_account_id
FROM "//home/cloud_analytics/emailing/sender/Go-to-paid-trial-usage"
UNION ALL
SELECT
    DISTINCT
        email,
        billing_account_id
FROM "//home/cloud_analytics/emailing/sender/Go-to-paid-stream"

FORMAT TabSeparatedWithNames
"""
old_df = lib.execute_query(req)

## Подготовка новой порции

In [17]:
new_table = main_df.copy()
new_table = new_table[
    (~new_table['email'].isin(marketo_prev_df['email'])) &
    (~new_table['billing_account_id'].isin(marketo_prev_df['billing_account_id']))
]
new_table = new_table[~new_table['email'].isin(dropped_emails)]

new_table = new_table[
    (~new_table['email'].isin(old_df['email'])) &
    (~new_table['billing_account_id'].isin(old_df['billing_account_id']))
]

new_table = new_table[
    (~new_table['email'].isin(upsell_df['email'])) &
    (~new_table['billing_account_id'].isin(upsell_df['billing_account_id']))
]
new_table = new_table[(new_table['is_isv'] == 0) &
                      (new_table['first_first_paid_consumption_datetime'] == '')]

new_table = new_table[(new_table['mail_feature'] == 1) |
                      (new_table['mail_info'] == 1) |
                      (new_table['mail_promo'] == 1)]
new_table = new_table[new_table['days_since_ba_created'] <= 14]

In [18]:
new_table['puid'] = new_table['puid'].astype(int)

In [19]:
new_table['Group'] = new_table['puid'].apply(
    lambda x: 'control' if x % 100 >= 35 and x % 100 <= 44 else 'test')

In [20]:
new_table['experiment_date'] = lib.get_current_date_as_str()
new_table['is_dropped'] = (new_table['email'].isin(dropped_emails)).astype(int)

In [21]:
new_table.shape

(358, 18)

In [22]:
lib.save_table("Go-to-paid-trial-usage", "//home/cloud_analytics/emailing/sender", 
               new_table, append=True)

In [23]:
def copy_after(table_name):
    if yt.row_count("//home/cloud_analytics/emailing/sender/" + table_name) != 0:
        yt.copy("//home/cloud_analytics/emailing/sender/" + table_name,
                "//home/cloud_analytics/emailing/sender_copy/" + table_name, force=True)
copy_after("Go-to-paid-trial-usage")
copy_after("Go-to-paid-trial-usage-OLD")

---

In [8]:
# marketo_previous_req = """
# SELECT
#     DISTINCT
#     billing_account_id,
#     any(email) as email,
#     groupUniqArray(mailing_name) as sended_mails,
#     'test' as Group
# FROM "//home/cloud_analytics/cubes/emailing_events/emailing_events"
# WHERE lower(mailing_name) like '%go-to-paid-trial-usage%'
# or lower(mailing_name) like '%q2-2020-upsell-1%'
# GROUP BY billing_account_id
# FORMAT TabSeparatedWithNames
# """
# marketo_prev_df = lib.execute_query(marketo_previous_req)
# marketo_prev_df['sended_mails'] = marketo_prev_df['sended_mails'].astype(str)
# lib.save_table("Go-to-paid-trial-usage-OLD", "//home/cloud_analytics/emailing/sender", 
#                marketo_prev_df)